In [1]:
import sys
import os
import re
import math
import glob
import uuid
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, CLIPSegProcessor, CLIPSegForImageSegmentation
from diffusers import StableDiffusionPipeline
from loguru import logger

/root/anaconda3/envs/lisa/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-12 17:29:37,937] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
device = 'cuda:1'
model_name = "runwayml/stable-diffusion-v1-5"
sd_model_path = '/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9'
pipe = StableDiffusionPipeline.from_pretrained(sd_model_path, torch_dtype=torch.float16)
pipe.to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.89it/s]


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.21.2",
  "_name_or_path": "/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
type(pipe)


diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline

In [7]:
# test clip embeding
from transformers import AutoTokenizer, CLIPTextModel

model = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")

inputs = tokenizer(["a photograph of an astronaut riding a horse"], padding=True, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled (EOS token) states

In [8]:
last_hidden_state.shape

torch.Size([1, 10, 768])

In [3]:
prompt = "a photograph of an astronaut riding a horse"
prompt_embeds, negative_prompt_embeds = pipe.encode_prompt(
            prompt,
            device,
            1,
            True,
            None,
            prompt_embeds=None,
            negative_prompt_embeds=None,
            lora_scale=None,
    )

In [4]:
prompt_embeds.shape
# negative_prompt_embeds.shape

torch.Size([1, 77, 768])

In [5]:
prompt_embeds.device
import torch
last_hidden_state = torch.rand([1, 23, 1024], device=prompt_embeds.device)

In [6]:
text = "a photograph of an astronaut riding a horse"
image_filename = os.path.join('image', str(uuid.uuid4())[0:8] + ".png")
image = pipe(prompt_embeds=last_hidden_state, guidance_scale=0).images[0]
image.save(image_filename)
# !mkdir image

  0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (23x1024 and 768x320)

In [5]:
text = "a photograph of an astronaut riding a horse"
image_filename = os.path.join('image', str(uuid.uuid4())[0:8] + ".png")
image = pipe(text).images[0]
image.save(image_filename)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:03<00:00, 15.49it/s]


In [6]:
type(image)

PIL.Image.Image

In [13]:
pipe.unet.config.sample_size

64